In [151]:
import pandas as pd 
import os 
import json
import re

# Get data from scraping from Kickstarter GraphQL endpoint

In [122]:
start = "/Users/ivankoh/Downloads/bt4222 scraped combined/"

with open(start + "0_44134_story.json", "r") as f:
    story_list1 = json.load(f)
with open(start + "0_44134_risk.json", "r") as f:
    risk_list1 = json.load(f)
with open(start + "44135_81404_story.json", "r") as f:
    story_list2 = json.load(f)
with open(start + "44135_81404_risk.json", "r") as f:
    risk_list2 = json.load(f)
with open(start + "81404_122106_story.json", "r") as f:
    story_list3 = json.load(f)
with open(start + "81404_122106_risk.json", "r") as f:
    risk_list3 = json.load(f)
with open(start + "162808_203510_story.json", "r") as f:
    story_list5 = json.load(f)
with open(start + "162808_203510_risk.json", "r") as f:
    risk_list5 = json.load(f)

story_list = story_list1 + story_list2
risk_list = risk_list1 + risk_list2

# Get data from Web Robots (200+k of daily scraping)

In [123]:
base = "/Users/ivankoh/Downloads/kickstarter data/"
filepaths = [base + f for f in os.listdir(base) if f.endswith('.csv')]
df_1 = pd.concat(map(pd.read_csv, filepaths))
df_2 = df_1.drop_duplicates(subset=['id'])
df_2['main_url'] = df_2['urls'].apply(lambda x: eval(x)['web']['project'])
print(len(df_1))
print(len(df_2))

df_robots = df_2

392244
203510


/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_65430/617038088.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['main_url'] = df_2['urls'].apply(lambda x: eval(x)['web']['project'])


# Get data from WebScraper.io Scraping

In [124]:
df_wy = pd.read_excel("/Users/ivankoh/Downloads/bt4222 scraped combined/kickstarter_by_categories.xlsx")
df_bandy = pd.read_csv("/Users/ivankoh/Downloads/bt4222 scraped combined/bandy.csv")
df_valentin = pd.read_csv("/Users/ivankoh/Downloads/bt4222 scraped combined/merged_ivan_valentin.csv")
df_radell = pd.read_csv("/Users/ivankoh/Downloads/bt4222 scraped combined/df1_radell.csv")

valentin_mapping = df_valentin[['ivan_index', 'valentin_index']]
print(len(df_wy))
df_wy = df_wy.drop_duplicates('Link-href')
print(len(df_wy))

/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_65430/234834254.py:4: DtypeWarning: Columns (36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df_radell = pd.read_csv("/Users/ivankoh/Downloads/bt4222 scraped combined/df1_radell.csv")


50300
47341


# Merge WebScraper.io dataset (Contains rewards feature) with GraphQL datasets (Contains story and risk features) 

In [125]:
df_bandy_join = df_robots.merge(df_bandy, on="id", how="left")
df_bandy_join['final_index'] = range(len(df_bandy_join))

if len(story_list) == len(risk_list):
    for i, item in enumerate(story_list):
        df_bandy_join.loc[df_bandy_join['final_index'] == i, ['story']] = story_list[i]
        df_bandy_join.loc[df_bandy_join['final_index'] == i, ['risk']] = risk_list[i]
else:
    print("check story and risk list")


In [126]:
df_radell['radell_index'] = range(len(df_radell))
df_radell = df_radell[(df_radell['radell_index'] >= 162808) & (df_radell['radell_index'] < 203510)]
df_radell = df_radell[['radell_index', 'id']]
radell_range = df_bandy_join[(df_bandy_join['final_index'] >= 162808) & (df_bandy_join['final_index'] < 203510)]
radell_merged = radell_range.merge(df_radell,on='id',how='inner')

print(len(radell_merged))

14732


In [127]:
radell_mapping = radell_merged[['final_index', 'radell_index']].sort_values('radell_index')
base_num = 162808

for idx, row in radell_mapping.iterrows():
    df_bandy_join.loc[df_bandy_join['final_index'] == row['final_index'], ['story']] = story_list5[row['radell_index'] - base_num]
    df_bandy_join.loc[df_bandy_join['final_index'] == row['final_index'], ['risk']] = risk_list5[row['radell_index'] - base_num]

In [128]:
valentin_mapping = valentin_mapping.sort_values('valentin_index')
base_num = 81404

for idx, row in valentin_mapping.iterrows():
    df_bandy_join.loc[df_bandy_join['final_index'] == row['ivan_index'], ['story']] = story_list3[row['valentin_index'] - base_num]
    df_bandy_join.loc[df_bandy_join['final_index'] == row['ivan_index'], ['risk']] = risk_list3[row['valentin_index'] - base_num]

In [129]:
# remove empty story and risk
print("number of empty story to be filled:", len(df_bandy_join[pd.isnull(df_bandy_join['story'])]))

df_bandy_join_rm_na = df_bandy_join[pd.notnull(df_bandy_join['story'])]
print("number of non-empty story:", len(df_bandy_join_rm_na))

number of empty story to be filled: 54329
number of non-empty story: 149181


In [130]:
df_wy['Link-href'] = df_wy['Link-href'].apply(lambda x: str(x)[:str(x).find("?ref")])
df_bandy_join_rm_na['main_url'] = df_bandy_join_rm_na['main_url'].apply(lambda x: str(x)[:str(x).find("?ref")])
final = df_wy.merge(df_bandy_join_rm_na, left_on="Link-href", right_on="main_url", how="left")

/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_65430/4062542141.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bandy_join_rm_na['main_url'] = df_bandy_join_rm_na['main_url'].apply(lambda x: str(x)[:str(x).find("?ref")])


In [168]:
print(len(final))
print(len(final.drop_duplicates('Link-href'))) # no more duplicates 

47341
47341


# Inspect Results

In [172]:
#number of missing story/risk
len(final[pd.isnull(final['story'])])

15852

In [173]:
#distribution of final data (excluding rows with missing story/risk)
(final[pd.notnull(final['story'])])['Status'].value_counts()

successful    21841
failed         8722
canceled        911
live             15
Name: Status, dtype: int64

In [171]:
# def get_slug(url):
#     try:
#         if url and not None:
#             return re.search('/projects/(.*)', url).group(1)
#         else:
#             return ""
#     except:
#         print("Error!")
#         print(url)


# print(final[pd.isnull(final['story'])].columns)
# print(final[pd.isnull(final['story'])]['Link-href'].iloc[1])
# final[pd.isnull(final['story'])]['Link-href'].apply(get_slug)

# Export

In [170]:
final.to_csv("kickstarter_data_merged.csv")